In [16]:
import os
import tensorflow as tf
import math
import numpy as np
import itertools


from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset
import hashlib
from tqdm import tqdm

TODO: As we want to use more tf records (we have them) add their paths here

In [2]:
filenames = ['/home/bcj/stanford/cs230/project/waymo_data/segment-10023947602400723454_1120_000_1140_000_with_camera_labels.tfrecord']

In [3]:
NUM_SAMPLES = 100

In [4]:
from waymo_open_dataset.utils.box_utils import is_within_box_3d

In [5]:
def prep_data(frame, num_points = 40000):
    mesh_vertices, instance_labels, semantic_labels, instance_bboxes = None, None, None, None
    
    # Transform frame to point cloud
    (range_images, camera_projections,
 range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(
    frame)
    points, cp_points = frame_utils.convert_range_image_to_point_cloud(
    frame,
    range_images,
    camera_projections,
    range_image_top_pose) 
    
    # 3d points in vehicle frame.
    points_all = np.concatenate(points, axis=0)
    
    # sub sample points
    if num_points < 0 :
        mesh_vertices = points_all
    else:
        sub_samp_points = points_all[np.random.choice(np.arange(points_all.shape[0]), num_points),:]
        mesh_vertices = sub_samp_points
    
    
    num_detected_objects = len(frame.laser_labels)
    
    instance_labels = np.zeros((len(mesh_vertices),))
    semantic_labels = np.zeros((len(mesh_vertices),))
    instance_bboxes = np.zeros((num_detected_objects, 7))
    
    # loop through each ground truth object
    for instance_id, detected_object in enumerate(frame.laser_labels):
        box = detected_object.box
        points_in_bbox = is_within_box_3d(point=tf.convert_to_tensor(mesh_vertices, dtype=tf.float32), box=tf.convert_to_tensor(np.array([
            box.center_x,
            box.center_y,
            box.center_z,
            box.width,
            box.length,
            box.height,
            box.heading,
        ]).reshape((1,7)), dtype=tf.float32)).numpy()
        
        
        # Do a check if we have all points
        if num_points < 0:
            print(np.sum(points_in_bbox > 0 ), detected_object.num_lidar_points_in_box)
            
        return mesh_vertices, instance_labels, semantic_labels, instance_bboxes
        
        
        
        
        
        
    
        
        
                       

# Run data generation pipeline

In [17]:
dataset = tf.data.TFRecordDataset(filenames, compression_type='')
num_samples_so_far = 0
batch_size_for_print = 10

BASE_DATA_DIR = "."

for data in tqdm(dataset):
    
    # Read data from tfrecord
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    
    scan_name = hashlib.md5(str(frame.context).encode()).hexdigest()
    print(scan_name)
    
    # process frame into data format we want
    mesh_vertices, instance_labels, semantic_labels, instance_bboxes = prep_data(frame, 40000)
    
    # save to npy file with given schema (i.e. frame.context.name_{file_type}.npy)
    FILENAME_TEMPLATE = "{scan_name}_{data_type}.npy"
    
    ## Write mesh verticies
    with open(FILENAME_TEMPLATE.format(scan_name=scan_name, data_type="vert"), 'wb') as f:
        np.save(f, mesh_vertices)
        
    ## Write instance labels
    with open(FILENAME_TEMPLATE.format(scan_name=scan_name, data_type="ins_label"), 'wb') as f:
        np.save(f, instance_labels)
        
    ## Write semantic labels
    with open(FILENAME_TEMPLATE.format(scan_name=scan_name, data_type="sem_label"), 'wb') as f:
        np.save(f, semantic_labels)
    
    ## Write instance_bboxes labels
    with open(FILENAME_TEMPLATE.format(scan_name=scan_name, data_type="bbox"), 'wb') as f:
        np.save(f, instance_bboxes)
    
    
    
    # Logging and stopping when we've sampled enough
    num_samples_so_far += 1
    if num_samples_so_far > NUM_SAMPLES:
        break
    
    if not (num_samples_so_far % batch_size_for_print):
        print("{}% of samples written".format(round(100.0*num_samples_so_far/NUM_SAMPLES, 2)))

0it [00:00, ?it/s]

654b2f55434f07e7759c5f435487b7bc


1it [00:00,  1.19it/s]

67421721497544c3083a0bb82c868f7f


2it [00:01,  1.22it/s]

...

99it [01:23,  1.34it/s]

c9008d4a55cb8e34ab77151bab9a186d


100it [01:24,  1.34it/s]

100.0% of samples written
faf867c616e823e2606d74e23bfb623b


100it [01:25,  1.17it/s]


In [20]:
! pwd

/home/bcj/stanford/cs230/project/orig_votenet/votenet_data
